In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import MSELoss

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data,gain=1.0)
        nn.init.normal_(m.bias.data, mean=0.0, std=1.0)

In [ ]:
class Net(nn.Module):
    def __init__(self, args, num_drug, num_disease, num_protein, num_sideeffect,g):
        super(Net, self).__init__()
        self.args = args 
        self.criterion = MSELoss(reduction='sum')

        # f0(v)
        #function for retrieving the initial embedding vector of u.
        #torch.Tensor is a multi-dimensional matrix containing elements of a*b
        self.drug_embedding = nn.Parameter(torch.Tensor(num_drug, args.d))
        self.protein_embedding = nn.Parameter(torch.Tensor(num_protein, args.d))
        self.disease_embedding = nn.Parameter(torch.Tensor(num_disease, args.d))
        self.sideeffect_embedding = nn.Parameter(torch.Tensor(num_sideeffect, args.d))
        
        # Wr
        # wr and br are edge type nn weights
        self.drug_drug_wr = nn.Linear(args.d, args.d)
        self.drug_chemical_wr = nn.Linear(args.d, args.d)
        self.drug_disease_wr = nn.Linear(args.d, args.d)
        self.drug_sideeffect_wr = nn.Linear(args.d, args.d)
        self.drug_protein_wr = nn.Linear(args.d, args.d)

        self.protein_protein_wr = nn.Linear(args.d, args.d)
        self.protein_sequence_wr = nn.Linear(args.d, args.d)
        self.protein_disease_wr = nn.Linear(args.d, args.d)
        self.protein_drug_wr = nn.Linear(args.d, args.d)

        self.disease_drug_wr = nn.Linear(args.d, args.d)
        self.disease_protein_wr = nn.Linear(args.d, args.d)

        self.sideeffect_drug_wr = nn.Linear(args.d, args.d)

        # W1
        #weight of nn at the node embedding update step.
        self.W1 = nn.Linear(2*args.d, args.d)

        # projection G and H
        self.drug_disease_G = nn.Parameter(torch.Tensor(args.d, args.k))
        self.drug_disease_H = nn.Parameter(torch.Tensor(args.d, args.k))

        self.drug_drug_G = nn.Parameter(torch.Tensor(args.d, args.k))

        self.drug_chemical_G = nn.Parameter(torch.Tensor(args.d, args.k))

        self.drug_sideeffect_G = nn.Parameter(torch.Tensor(args.d, args.k))
        self.drug_sideeffect_H = nn.Parameter(torch.Tensor(args.d, args.k))

        self.drug_protein_G = nn.Parameter(torch.Tensor(args.d, args.k))
        self.drug_protein_H = nn.Parameter(torch.Tensor(args.d, args.k))

        self.protein_disease_G = nn.Parameter(torch.Tensor(args.d, args.k))
        self.protein_disease_H = nn.Parameter(torch.Tensor(args.d, args.k))

        self.protein_protein_G = nn.Parameter(torch.Tensor(args.d, args.k))

        self.protein_sequence_G = nn.Parameter(torch.Tensor(args.d, args.k))

        self.reset_parameters()
        self.apply(init_weights)
        
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.drug_disease_G, gain=1.0)
        nn.init.xavier_uniform_(self.drug_disease_H, gain=1.0)
        nn.init.xavier_uniform_(self.drug_drug_G, gain=1.0)
        nn.init.xavier_uniform_(self.drug_chemical_G, gain=1.0)  
        nn.init.xavier_uniform_(self.drug_sideeffect_G, gain=1.0)
        nn.init.xavier_uniform_(self.drug_sideeffect_H, gain=1.0) 
        nn.init.xavier_uniform_(self.drug_protein_G, gain=1.0)
        nn.init.xavier_uniform_(self.drug_protein_H, gain=1.0)
        nn.init.xavier_uniform_(self.protein_disease_G, gain=1.0)
        nn.init.xavier_uniform_(self.protein_disease_H, gain=1.0)
        nn.init.xavier_uniform_(self.protein_protein_G, gain=1.0)
        nn.init.xavier_uniform_(self.protein_sequence_G, gain=1.0)
        nn.init.xavier_uniform_(self.drug_embedding, gain=1.0)
        nn.init.xavier_uniform_(self.protein_embedding, gain=1.0)
        nn.init.xavier_uniform_(self.disease_embedding, gain=1.0)
        nn.init.xavier_uniform_(self.sideeffect_embedding, gain=1.0)
    
    def forward(self, drug_drug_normalize, drug_chemical_normalize, drug_disease_normalize, drug_sideeffect_normalize, protein_protein_normalize, protein_sequence_normalize, protein_disease_normalize, 
    disease_drug_normalize, disease_protein_normalize, sideeffect_drug_normalize, drug_protein_normalize, protein_drug_normalize, drug_drug, drug_chemical, drug_disease, drug_sideeffect, protein_protein, protein_sequence, protein_disease, 
    drug_protein, drug_protein_mask):
        # passing 1 times (can be easily extended to multiple passes)
        #F.normalize performs Lp normalization of inputs
        drug_vector1 = F.normalize(torch.sigmoid(self.W1(torch.cat([
            torch.matmul(drug_drug_normalize, self.drug_drug_wr(self.drug_embedding)) + \
            torch.matmul(drug_chemical_normalize, self.drug_chemical_wr(self.drug_embedding)) + \
            torch.matmul(drug_disease_normalize, self.drug_disease_wr(self.disease_embedding)) + \
            torch.matmul(drug_sideeffect_normalize, self.drug_sideeffect_wr(self.sideeffect_embedding)) + \
            torch.matmul(drug_protein_normalize, self.drug_protein_wr(self.protein_embedding)), 
            self.drug_embedding
        ], dim=1))))

        protein_vector1 = F.normalize(torch.sigmoid(self.W1(torch.cat([
            torch.matmul(protein_protein_normalize, self.protein_protein_wr(self.protein_embedding)) + \
            torch.matmul(protein_sequence_normalize, self.protein_sequence_wr(self.protein_embedding)) + \
            torch.matmul(protein_disease_normalize, self.protein_disease_wr(self.disease_embedding)) + \
            torch.matmul(protein_drug_normalize, self.protein_drug_wr(self.drug_embedding)), 
            self.protein_embedding
        ], dim=1))))

        disease_vector1 = F.normalize(torch.sigmoid(self.W1(torch.cat([
            torch.matmul(disease_drug_normalize, self.disease_drug_wr(self.drug_embedding)) + \
            torch.matmul(disease_protein_normalize, self.disease_protein_wr(self.protein_embedding)),
            self.disease_embedding
        ], dim=1))))

        sideeffect_vector1 = F.normalize(torch.sigmoid(self.W1(torch.cat([
            torch.matmul(sideeffect_drug_normalize, self.sideeffect_drug_wr(self.drug_embedding)),
            self.sideeffect_embedding
        ], dim=1))))
        # print(protein_vector1, disease_vector1, drug_vector1, sideeffect_vector1)
        # reconstructing networks

        #torch.linalg.multi_dot multiplies multiple matrices.
        drug_drug_reconstruct = torch.linalg.multi_dot([drug_vector1, self.drug_drug_G, self.drug_drug_G.T, drug_vector1.T])
        drug_drug_reconstruct_loss = self.criterion(drug_drug_reconstruct, drug_drug)
        
        drug_chemical_reconstruct = torch.linalg.multi_dot([drug_vector1, self.drug_chemical_G, self.drug_chemical_G.T, drug_vector1.T])
        drug_chemical_reconstruct_loss = self.criterion(drug_chemical_reconstruct, drug_chemical)

        drug_disease_reconstruct = torch.linalg.multi_dot([drug_vector1, self.drug_disease_G, self.drug_disease_H.T, disease_vector1.T])
        drug_disease_reconstruct_loss = self.criterion(drug_disease_reconstruct, drug_disease)

        drug_sideeffect_reconstruct = torch.linalg.multi_dot([drug_vector1, self.drug_sideeffect_G, self.drug_sideeffect_H.T, sideeffect_vector1.T])
        drug_sideeffect_reconstruct_loss = self.criterion(drug_sideeffect_reconstruct, drug_sideeffect)

        protein_protein_reconstruct = torch.linalg.multi_dot([protein_vector1, self.protein_protein_G, self.protein_protein_G.T, protein_vector1.T])
        protein_protein_reconstruct_loss = self.criterion(protein_protein_reconstruct, protein_protein)

        protein_sequence_reconstruct = torch.linalg.multi_dot([protein_vector1, self.protein_sequence_G, self.protein_sequence_G.T, protein_vector1.T])
        protein_sequence_reconstruct_loss = self.criterion(protein_sequence_reconstruct, protein_sequence)
        
        protein_disease_reconstruct = torch.linalg.multi_dot([protein_vector1, self.protein_disease_G, self.protein_disease_H.T, disease_vector1.T])
        protein_disease_reconstruct_loss  = self.criterion(protein_disease_reconstruct, protein_disease)

        drug_protein_reconstruct = torch.linalg.multi_dot([drug_vector1, self.drug_protein_G, self.drug_protein_H.T, protein_vector1.T])
        tmp = torch.mul(drug_protein_mask, (drug_protein_reconstruct - drug_protein))
        drug_protein_reconstruct_loss = torch.sum(tmp.pow(2))
            
        other_loss = drug_drug_reconstruct_loss + drug_chemical_reconstruct_loss + drug_disease_reconstruct_loss + drug_sideeffect_reconstruct_loss + protein_protein_reconstruct_loss + protein_sequence_reconstruct_loss + protein_disease_reconstruct_loss       

    
        total_loss = drug_protein_reconstruct_loss + 1.0 * (other_loss)
        
        return total_loss, drug_protein_reconstruct_loss, drug_protein_reconstruct.detach().cpu().numpy()

In [ ]:
import numpy as np 
import torch, os, argparse, random
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

In [ ]:
import easydict

def get_args():
  args = easydict.EasyDict({
     "seed": 26, #random seed for init.
     "d": 1024, #the embedding dim.
     "n":1.0, #global gradient norm
     "k":512, #the dim. of reprojection matrices k.
     "t":"o", #the dti matrix for testing, o -> mat_drug_protein.txt.
     "r":"ten", #positive-negative ratio.
     "lr":1e-3, #learning rate
     "weight_decay":60, #weight decay of the optimizer
     "num_steps":100, #no. of forward propg.
     "device":"cuda", # device type.
     "n_folds":3, #number of folds for cross vld.
     "round":1, # number of rounds of sampling
     "test_size":0.05 # portion of vld. data.
})
  return args

In [ ]:
def row_normalize(a_matrix, substract_self_loop):
    if substract_self_loop == True:
        np.fill_diagonal(a_matrix,0)
    a_matrix = a_matrix.astype(float)
    row_sums = a_matrix.sum(axis=1)+1e-12
    new_matrix = a_matrix / row_sums[:, np.newaxis]
    new_matrix[np.isnan(new_matrix) | np.isinf(new_matrix)] = 0.0
    return torch.Tensor(new_matrix)

In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.9 MB/s 


In [ ]:
import dgl

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [ ]:
def train_and_evaluate(args, DTItrain, DTIvalid, DTItest, verbose=True):
    set_seed(args)
    drug_protein = np.zeros((num_drug,num_protein))
    mask = np.zeros((num_drug,num_protein))
    for ele in DTItrain:
        drug_protein[ele[0],ele[1]] = ele[2]
        mask[ele[0],ele[1]] = 1
    protein_drug = drug_protein.T

    drug_protein_normalize = row_normalize(drug_protein,False).to(device)
    protein_drug_normalize = row_normalize(protein_drug,False).to(device)
    drug_protein = torch.Tensor(drug_protein).to(device)
    
    mask = torch.Tensor(mask).to(device)
    model = Net(args, num_drug, num_disease, num_protein, num_sideeffect,g)
    model.to(device)
    no_decay = ["bias"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.8, patience=2)

    ground_truth_train = [ele[2] for ele in DTItrain]
    ground_truth_valid = [ele[2] for ele in DTIvalid]
    ground_truth_test = [ele[2] for ele in DTItest]


    best_valid_aupr = 0
    best_valid_auc = 0
    test_aupr = 0
    test_auc = 0
    for i in range(args.num_steps):
        model.train()
        model.zero_grad()
        tloss, dtiloss, results = model(drug_drug_normalize, drug_chemical_normalize, drug_disease_normalize, 
                                        drug_sideeffect_normalize, protein_protein_normalize, protein_sequence_normalize, 
                                        protein_disease_normalize, disease_drug_normalize, disease_protein_normalize, 
                                        sideeffect_drug_normalize, drug_protein_normalize, protein_drug_normalize, 
                                        drug_drug, drug_chemical, drug_disease, drug_sideeffect, protein_protein, 
                                        protein_sequence, protein_disease, drug_protein, mask)
        # print(results)
        tloss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.n)
        optimizer.step()
        if i % 25 == 0 and verbose == True:
            print('step', i, 'total and dti loss', tloss.item(), dtiloss.item())
            model.eval()
            pred_list_valid = [results[ele[0],ele[1]] for ele in DTIvalid]
            valid_auc = roc_auc_score(ground_truth_valid, pred_list_valid)
            valid_aupr = average_precision_score(ground_truth_valid, pred_list_valid)

            pred_list_train = [results[ele[0],ele[1]] for ele in DTItrain]
            train_auc = roc_auc_score(ground_truth_train, pred_list_train)
            train_aupr = average_precision_score(ground_truth_train, pred_list_train)
            scheduler.step(train_aupr)
            if valid_aupr >= best_valid_aupr:
                best_valid_aupr = valid_aupr
                best_valid_auc = valid_auc
                pred_list_test = [results[ele[0],ele[1]] for ele in DTItest]
                test_auc = roc_auc_score(ground_truth_test, pred_list_test)
                test_aupr = average_precision_score(ground_truth_test, pred_list_test)
            print ('train auc aupr', train_auc, train_aupr, 'valid auc aupr,', valid_auc, valid_aupr, 'test auc aupr', test_auc, test_aupr)
        
    return best_valid_auc, best_valid_aupr, test_auc, test_aupr

if __name__ == '__main__':
    args = get_args()
    set_seed(args)
    device = torch.device(args.device)
    network_path = '/content/drive/MyDrive/Colab Notebooks/data/'
    print('loading networks ...')
    drug_drug = np.loadtxt(network_path+'mat_drug_drug.txt')
    true_drug = 708 # First [0:708] are drugs, the rest are compounds retrieved from ZINC15 database
    drug_chemical = np.loadtxt(network_path+'Similarity_Matrix_Drugs.txt')
    drug_chemical=drug_chemical[:true_drug,:true_drug]
    drug_disease = np.loadtxt(network_path+'mat_drug_disease.txt')
    drug_sideeffect = np.loadtxt(network_path+'mat_drug_se.txt')
    disease_drug = drug_disease.T
    sideeffect_drug = drug_sideeffect.T

    protein_protein = np.loadtxt(network_path+'mat_protein_protein.txt')
    protein_sequence = np.loadtxt(network_path+'Similarity_Matrix_Proteins.txt')
    protein_disease = np.loadtxt(network_path+'mat_protein_disease.txt')
    disease_protein = protein_disease.T

    def ConstructGraph(drug_drug, drug_chemical, drug_disease, drug_sideeffect, protein_protein, protein_sequence,
                   protein_disease):
      num_drug = len(drug_drug)
      num_protein = len(protein_protein)
      num_disease = len(drug_disease.T)
      num_sideeffect = len(drug_sideeffect.T)

      list_drug = []
      for i in range(num_drug):
          list_drug.append((i, i))

      list_protein = []
      for i in range(num_protein):
          list_protein.append((i, i))

      list_disease = []
      for i in range(num_disease):
          list_disease.append((i, i))

      list_sideeffect = []
      for i in range(num_sideeffect):
          list_sideeffect.append((i, i))

      list_DDI = []
      for row in range(num_drug):
          for col in range(num_drug):
              if drug_drug[row, col] > 0:
                  list_DDI.append((row, col))

      list_PPI = []
      for row in range(num_protein):
          for col in range(num_protein):
              if protein_protein[row, col] > 0:
                  list_PPI.append((row, col))

      list_drug_sideeffect = []
      list_sideeffect_drug = []
      for row in range(num_drug):
          for col in range(num_sideeffect):
              if drug_sideeffect[row, col] > 0:
                  list_drug_sideeffect.append((row, col))
                  list_sideeffect_drug.append((col, row))

      list_drug_disease = []
      list_disease_drug = []
      for row in range(num_drug):
          for col in range(num_disease):
              if drug_disease[row, col] > 0:
                  list_drug_disease.append((row, col))
                  list_disease_drug.append((col, row))

      list_protein_disease = []
      list_disease_protein = []
      for row in range(num_protein):
          for col in range(num_disease):
              if protein_disease[row, col] > 0:
                  list_protein_disease.append((row, col))
                  list_disease_protein.append((col, row))

      g_HIN = dgl.heterograph({('disease', 'disease_disease virtual', 'disease'): list_disease,
                             ('drug', 'drug_drug virtual', 'drug'): list_drug,
                             ('protein', 'protein_protein virtual', 'protein'): list_protein,
                             ('sideeffect', 'sideeffect_sideeffect virtual', 'sideeffect'): list_sideeffect,
                             ('drug', 'drug_drug interaction', 'drug'): list_DDI, \
                             ('protein', 'protein_protein interaction', 'protein'): list_PPI, \
                             ('drug', 'drug_sideeffect association', 'sideeffect'): list_drug_sideeffect, \
                             ('sideeffect', 'sideeffect_drug association', 'drug'): list_sideeffect_drug, \
                             ('drug', 'drug_disease association', 'disease'): list_drug_disease, \
                             ('disease', 'disease_drug association', 'drug'): list_disease_drug, \
                             ('protein', 'protein_disease association', 'disease'): list_protein_disease, \
                             ('disease', 'disease_protein association', 'protein'): list_disease_protein})

      g = g_HIN.edge_type_subgraph(['drug_drug interaction', 'protein_protein interaction',
                                  'drug_sideeffect association', 'sideeffect_drug association',
                                  'drug_disease association', 'disease_drug association',
                                  'protein_disease association', 'disease_protein association'
                                  ])

      return g
      
    
    print('normalize network for mean pooling aggregation')
    drug_drug_normalize = row_normalize(drug_drug,True).to(device)
    drug_chemical_normalize = row_normalize(drug_chemical,True).to(device)
    drug_disease_normalize = row_normalize(drug_disease,False).to(device)
    drug_sideeffect_normalize = row_normalize(drug_sideeffect,False).to(device)

    protein_protein_normalize = row_normalize(protein_protein,True).to(device)
    protein_sequence_normalize = row_normalize(protein_sequence,True).to(device)
    protein_disease_normalize = row_normalize(protein_disease,False).to(device)

    disease_drug_normalize = row_normalize(disease_drug,False).to(device)
    disease_protein_normalize = row_normalize(disease_protein,False).to(device)
    sideeffect_drug_normalize = row_normalize(sideeffect_drug,False).to(device)

    

    #define computation graph

    g = ConstructGraph(drug_drug, drug_chemical, drug_disease, drug_sideeffect, protein_protein, protein_sequence,
                       protein_disease)

    num_drug = len(drug_drug_normalize)
    num_protein = len(protein_protein_normalize)
    num_disease = len(disease_protein_normalize)
    num_sideeffect = len(sideeffect_drug_normalize)

    drug_drug = torch.Tensor(drug_drug).to(device)
    drug_chemical = torch.Tensor(drug_chemical).to(device)
    drug_disease = torch.Tensor(drug_disease).to(device)
    drug_sideeffect = torch.Tensor(drug_sideeffect).to(device)
    protein_protein = torch.Tensor(protein_protein).to(device)
    protein_sequence = torch.Tensor(protein_sequence).to(device)
    protein_disease = torch.Tensor(protein_disease).to(device)

    # prepare drug_protein and mask
    test_auc_round = []
    test_aupr_round = []
    val_auc_round = []
    val_aupr_round = []


    if args.t == 'o':
        dti_o = np.loadtxt(network_path+'mat_drug_protein.txt')
    else:
        dti_o = np.loadtxt(network_path+'mat_drug_protein_'+args.t+'.txt')

    whole_positive_index = []
    whole_negative_index = []
    for i in range(np.shape(dti_o)[0]):
        for j in range(np.shape(dti_o)[1]):
            if int(dti_o[i][j]) == 1:
                whole_positive_index.append([i,j])
            elif int(dti_o[i][j]) == 0:
                whole_negative_index.append([i,j])

    for r in range(args.round):
        print ('sample round',r+1)
        if args.r == 'ten':
            negative_sample_index = np.random.choice(np.arange(len(whole_negative_index)),size=10*len(whole_positive_index),replace=False)
        elif args.r == 'all':
            negative_sample_index = np.arange(len(whole_negative_index))
        else:
            print ('wrong positive negative ratio')
            break

        data_set = np.zeros((len(negative_sample_index)+len(whole_positive_index),3),dtype=int)
        count = 0
        for i in whole_positive_index:
            data_set[count][0] = i[0]
            data_set[count][1] = i[1]
            data_set[count][2] = 1
            count += 1
        for i in negative_sample_index:
            data_set[count][0] = whole_negative_index[i][0]
            data_set[count][1] = whole_negative_index[i][1]
            data_set[count][2] = 0
            count += 1



        if args.t == 'unique':
            whole_positive_index_test = []
            whole_negative_index_test = []
            for i in range(np.shape(dti_o)[0]):
                for j in range(np.shape(dti_o)[1]):
                    if int(dti_o[i][j]) == 3:
                        whole_positive_index_test.append([i,j])
                    elif int(dti_o[i][j]) == 2:
                        whole_negative_index_test.append([i,j])

            if args.r == 'ten':
                negative_sample_index_test = np.random.choice(np.arange(len(whole_negative_index_test)),size=10*len(whole_positive_index_test),replace=False)
            elif args.r == 'all':
                negative_sample_index_test = np.arange(len(whole_negative_index_test))
            else:
                print ('wrong positive negative ratio')
                break
            data_set_test = np.zeros((len(negative_sample_index_test)+len(whole_positive_index_test),3),dtype=int)
            count = 0
            for i in whole_positive_index_test:
                data_set_test[count][0] = i[0]
                data_set_test[count][1] = i[1]
                data_set_test[count][2] = 1
                count += 1
            for i in negative_sample_index_test:
                data_set_test[count][0] = whole_negative_index_test[i][0]
                data_set_test[count][1] = whole_negative_index_test[i][1]
                data_set_test[count][2] = 0
                count += 1

            DTItrain = data_set
            DTItest = data_set_test
            rs = np.random.randint(0,1000,1)[0]
            DTItrain, DTIvalid =  train_test_split(DTItrain, test_size=args.test_size, random_state=rs)
            v_auc, v_aupr, t_auc, t_aupr = train_and_evaluate(DTItrain=DTItrain, DTIvalid=DTIvalid, DTItest=DTItest)

            test_auc_round.append(t_auc)
            test_aupr_round.append(t_aupr)
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/test_auc.csv', test_auc_round,fmt="%.4f")
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/test_aupr.csv', test_aupr_round,fmt="%.4f")

        else:
            val_auc_fold = []
            val_aupr_fold = []
            test_auc_fold = []
            test_aupr_fold = []
            rs = np.random.randint(0,1000,1)[0]
            skf = StratifiedKFold(n_splits=args.n_folds, shuffle=True, random_state=rs)

            for train_index, test_index in skf.split(np.arange(len(data_set)), data_set[:,2]):
                DTItrain, DTItest = data_set[train_index], data_set[test_index]
                DTItrain, DTIvalid =  train_test_split(DTItrain, test_size=args.test_size, random_state=rs)

                v_auc, v_aupr, t_auc, t_aupr = train_and_evaluate(args=args, DTItrain=DTItrain, DTIvalid=DTIvalid, DTItest=DTItest)
                val_auc_fold.append(v_auc)
                val_aupr_fold.append(v_aupr)
                test_auc_fold.append(t_auc)
                test_aupr_fold.append(t_aupr)
                # break
            val_auc_round.append(np.mean(val_auc_fold))
            val_aupr_round.append(np.mean(val_aupr_fold))
            test_auc_round.append(np.mean(test_auc_fold))
            test_aupr_round.append(np.mean(test_aupr_fold))
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/val_auc.csv', val_auc_round,fmt="%.4f")
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/val_aupr.csv', val_aupr_round,fmt="%.4f")
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/test_auc.csv', test_auc_round,fmt="%.4f")
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/data/test_aupr.csv', test_aupr_round,fmt="%.4f")

loading networks ...
normalize network for mean pooling aggregation
sample round 1
step 0 total and dti loss 317056128.0 1287.08984375
train auc aupr 0.1712802815882898 0.05416426530429627 valid auc aupr, 0.31729571737196594 0.059582199874294216 test auc aupr 0.28361399042545166 0.05884562554454195
step 25 total and dti loss 339180160.0 1229.71484375
train auc aupr 0.4598926049439397 0.0789560914096818 valid auc aupr, 0.5029355699580633 0.09016132530164629 test auc aupr 0.5085235141074909 0.08833614041580662
step 50 total and dti loss 346201184.0 1221.998291015625
train auc aupr 0.7453475540128518 0.15271883148173226 valid auc aupr, 0.6947515567416445 0.13953414723406193 test auc aupr 0.6959108598353293 0.13627064475737677
step 75 total and dti loss 347450144.0 1221.28271484375
train auc aupr 0.4825095395937285 0.08328413367571721 valid auc aupr, 0.4540856525606811 0.07694935966394334 test auc aupr 0.6959108598353293 0.13627064475737677
step 0 total and dti loss 317047264.0 1286.922119